In [357]:
import numpy as np
import pandas as pd
import os
import re
from tqdm import tqdm
import ast
import scipy.cluster.hierarchy as shc
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np
import string
from IPython.display import Markdown
from collections import Counter
from fonctions import *
import textdistance

In [101]:
def test_2(sentence1,sentence2):
    
    texte1 = remove_characters(sentence1,characters_to_remove).lower().lstrip().rstrip().replace('j','i').replace('v','u')
    pierre1 = remove_characters(sentence2,characters_to_remove).lower().lstrip().rstrip().replace('j','i').replace('v','u')

    texte_tri = string_to_trigrams(texte1.split())
    pierre_tri = string_to_trigrams(pierre1.split())

    liste_trigrams = []
    for i in range(len(pierre_tri)):
        if pierre_tri[i] in texte_tri:
            liste_trigrams.append(pierre_tri[i])
    indices_trigrams = []
    for i in range(len(pierre_tri)):
        if pierre_tri[i] in texte_tri:
            indices_trigrams.append(i)


    index = get_first_numbers(indices_trigrams)

    texte_split = texte1.split()
    pierre_split = pierre1.split()

    index_texte = []
    index_pierre = []


    for i in index:

        trigram = pierre_tri[i]

        x = ' '.join(trigram)
        texte = find_trigram_indices(x,texte1)
        pierre = find_trigram_indices(x,pierre1)

        taille_texte = len(texte_split)
        taille_pierre = len(pierre_split)

        i = -1

        while texte[i]+1 < taille_texte -1 and pierre[i]+1 < taille_pierre -1:
            if texte_split[texte[i]+1]  == pierre_split[pierre[i]+1]:
                texte.append(texte[-1]+1) 
                pierre.append(pierre[-1]+1)
            elif 0.8 <= textdistance.levenshtein.normalized_similarity(texte_split[texte[i]+1],pierre_split[pierre[i]+1]) < 1:
                texte.append(texte[-1]+1) 
                pierre.append(pierre[-1]+1)
            else:
                break
                
        index_pierre.extend(pierre)
        index_texte.extend(texte)
        
    for i in index_pierre:
        pierre_split[i] = f'**{pierre_split[i]}**'
    for i in index_texte:
        texte_split[i] = f'**{texte_split[i]}**'
        
        #Insérer ici
        

    pierre_indice = []
    texte_indice= []

    pierre_mot = []
    texte_mot = []

    for idx1, word1 in enumerate(texte_split):
        for idx2, word2 in enumerate(pierre_split):
            similarity = textdistance.levenshtein.normalized_similarity(word1, word2)
            if 0.8 <= similarity < 1 and word1 in texte_split and word2 in pierre_split:
                pierre_indice.append(idx2)
                texte_indice.append(idx1)
                pierre_mot.append(word2)
                texte_mot.append(word1)


    tex = []
    pie = []
    for i in range(len(pierre_indice)):

        inter_tex = []
        inter_pie = []

        if texte_indice[i] == taille_texte-1 and texte_split[texte_indice[i]-1] == pierre_split[pierre_indice[i]-1] and texte_split[texte_indice[i]-2] == pierre_split[pierre_indice[i]-2]:

            inter_tex.append(texte_indice[i])
            inter_pie.append(pierre_indice[i]) 
            value = 1
            while texte_split[texte_indice[i]-value] == pierre_split[pierre_indice[i]-value]:
                inter_tex.append(texte_indice[i]-value)
                inter_pie.append(pierre_indice[i]-value)
                value+=1

        elif texte_indice[i] == taille_texte -2 and texte_split[texte_indice[i] + 1] == pierre_split[pierre_indice[i] +1 ] and texte_split[texte_indice[i]-1] == pierre_split[pierre_indice[i]-1]:
            inter_tex.append(texte_indice[i]+1)
            inter_tex.append(texte_indice[i])
            inter_pie.append(pierre_indice[i]+1)
            inter_pie.append(pierre_indice[i])

            value = 1
            while texte_split[texte_indice[i]-value] == pierre_split[pierre_indice[i]-value]:
                inter_tex.append(texte_indice[i]-value)
                inter_pie.append(pierre_indice[i]-value)
                value+=1 

        elif texte_indice[i] == 0 and texte_split[texte_indice[i]+1] == pierre_split[pierre_indice[i]+1] and texte_split[texte_indice[i]+2] == pierre_split[pierre_indice[i]+2]:
            inter_tex.append(texte_indice[i])
            inter_pie.append(pierre_indice[i])

            value = 1
            while texte_split[texte_indice[i]+value] == pierre_split[pierre_indice[i]+value]:
                inter_tex.append(texte_indice[i]+value)
                inter_pie.append(pierre_indice[i]+value)
                value+=1

        elif texte_indice[i] == 1 and texte_split[texte_indice[i]-1] == pierre_split[pierre_indice[i]-1] and texte_split[texte_indice[i]+1] == pierre_split[pierre_indice[i]+1]:
            inter_tex.append(texte_indice[i]-1)
            inter_tex.append(texte_indice[i])
            inter_pie.append(pierre_indice[i]-1)
            inter_pie.append(pierre_indice[i])

            value = 1
            while texte_split[texte_indice[i]+value] == pierre_split[pierre_indice[i]+value]:
                inter_tex.append(texte_indice[i]+value)
                inter_pie.append(pierre_indice[i]+value)
                value+=1


        elif texte_indice[i]+ 2 < taille_texte and pierre_indice[i]+2 < taille_pierre and texte_split[texte_indice[i]+1] == pierre_split[pierre_indice[i]+1] and texte_split[texte_indice[i]-1] != pierre_split[pierre_indice[i]-1] and texte_split[texte_indice[i]+2] == pierre_split[pierre_indice[i]+2]:
            inter_tex.append(texte_indice[i])
            inter_pie.append(pierre_indice[i])
            value = 1
            while texte_indice[i]+value < taille_texte and pierre_indice[i]+value < taille_pierre and texte_split[texte_indice[i]+value] == pierre_split[pierre_indice[i]+value]:
                inter_tex.append(texte_indice[i]+value)
                inter_pie.append(pierre_indice[i]+value)
                value+=1
                
        elif texte_split[texte_indice[i]-1] == pierre_indice[i]-1 and texte_split[texte_indice[i]+1] != pierre_split[pierre_indice[i]+1] and texte_split[texte_indice[i]-2] == pierre_split[pierre_indice[i]-2]:
            inter_tex.append(texte_indice[i])
            inter_pie.append(pierre_indice[i])
            value = 1
            while texte_indice[i]+value < taille_texte and pierre_indice[i]+value < taille_pierre and texte_split[texte_indice[i]-value] == pierre_split[pierre_indice[i]-value]:
                inter_tex.append(texte_indice[i]-value)
                inter_pie.append(pierre_indice[i]-value)
                value+=1

        elif texte_split[texte_indice[i]-1] == pierre_indice[i]-1 and texte_split[texte_indice[i]+1] == pierre_split[pierre_indice[i]+1]:
            inter_tex.append(texte_indice[i]-1)
            inter_pie.append(pierre_indice[i]-1)
            inter_tex.append(texte_indice[i])
            inter_pie.append(pierre_indice[i])        
            inter_tex.append(texte_indice[i]+1)
            inter_pie.append(pierre_indice[i]+1)

            value = 2
            while texte_indice[i]+value < taille_texte and pierre_indice[i]+value < taille_pierre and texte_split[texte_indice[i]-value] == pierre_split[pierre_indice[i]-value]:
                inter_tex.insert(0,texte_indice[i]-value)
                inter_pie.insert(0,pierre_indice[i]-value)
                value+=1
            value = 2
            while texte_indice[i]+value < taille_texte and pierre_indice[i]+value < taille_pierre and texte_split[texte_indice[i]+value] == pierre_split[pierre_indice[i]+value]:
                inter_tex.insert(0,texte_indice[i]+value)
                inter_pie.insert(0,pierre_indice[i]+value)
                value +=1

        tex.extend(inter_tex)   
        pie.extend(inter_pie)
        
    unique_texte = [value for value in tex if value not in index_texte]
    unique_pierre = [value for value in pie if value not in index_pierre]
    
    for i in unique_pierre:
        pierre_split[i] = f'**{pierre_split[i]}**'
    for i in unique_texte:
        texte_split[i] = f'**{texte_split[i]}**'
   

    texte_final = ' '.join(texte_split)

    pierre_final = ' '.join(pierre_split)


    return [transformer_asterisques(texte_split), transformer_asterisques(pierre_split)]

In [109]:
def find_trigram_indices(trigram, phrase):
    words = phrase.split()
    trigram_indices = []

    for i in range(len(words) - 2):
        if ' '.join(words[i:i+3]) == trigram:
            trigram_indices.extend([i, i+1, i+2])
    return trigram_indices

In [103]:
def remove_characters(input_string, characters_to_remove):
    translator = str.maketrans('', '', characters_to_remove)
    cleaned_string = input_string.translate(translator)
    return cleaned_string

In [105]:
characters_to_remove = string.punctuation + '[]' +'–'

In [107]:
def get_first_numbers(numbers):
    result = []
    current_sequence = []

    for num in numbers:
        if not current_sequence or num == current_sequence[-1] + 1:
            current_sequence.append(num)
        else:
            result.append(current_sequence[0])
            current_sequence = [num]

    if current_sequence:
        result.append(current_sequence[0])

    return result

In [113]:
def transformer_asterisques(liste):
    nouvelle_liste = []
    
    for element in liste:
        nouveau_element = element.replace("****", "**", 1).replace("****", "**", 1)
        nouvelle_liste.append(nouveau_element)
    
    return nouvelle_liste

In [83]:
def flatten_tri(liste):
    return [trigram for sublist in liste for trigram in sublist]

In [73]:
def copies(liste):
    nouvelles_copies = []

    for element in liste:
        nouvelles_copies.extend([element] * len(dico_un))

    return nouvelles_copies

In [76]:
def avoir_trigram_commun(phrase1, phrase2):
    trigrams_phrase1 = set(tuple(phrase1.split()[i:i+3]) for i in range(len(phrase1.split()) - 2))
    trigrams_phrase2 = set(tuple(phrase2.split()[i:i+3]) for i in range(len(phrase2.split()) - 2))
    
    return bool(trigrams_phrase1 & trigrams_phrase2)

In [79]:
def string_to_trigrams(input_string):
    trigrams = [input_string[i:i+3] for i in range(len(input_string) - 2)]
    return trigrams

In [71]:
file = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/PDV_grams.xlsx'
viard = '/home/noe/Documents/2022-2023/ENC/Mémoire/Dataframes/Viard_grams.xlsx'

def delete_items_with_prefix(dictionnaire, prefix):
    keys_to_delete = [key for key in dictionnaire if key.startswith(prefix)]
    for key in keys_to_delete:
        del dictionnaire[key]


def is_single_word(s):
    return isinstance(s, str) and len(s.split()) == 1

def keys_with_single_word_value(dictionary):
    single_word_keys = []
    for key, value in dictionary.items():
        if is_single_word(value):
            single_word_keys.append(key)
    return single_word_keys
def join(liste):
    return ' '.join(liste)

df_pdv = pd.read_excel(file)
df_viard = pd.read_excel(viard)

df = pd.DataFrame({'texts':df_pdv['textes'],'clean_PDV': df_pdv['textes'].apply(clean_text)})
df['compare'] = df['clean_PDV'].apply(join)

df2 = pd.DataFrame({'viard_texts':df_viard['text'],'clean_viard':df_viard['text'].apply(clean_text)})

column_without_brackets = [[string.replace('[', '').replace(']', '').lstrip().rstrip() for string in sublist] for sublist in df2['clean_viard']]

df2['clean'] = column_without_brackets
df2['compare'] = df2['clean'].apply(join)
viard = {}
for i in range(len(df2['compare'].tolist())):
    viard[f'viard_{i+1}'] = df2['compare'].tolist()[i]

In [70]:
dico_un = {}

for i in range(len(df['compare'].tolist())):
    dico_un[f'pdv_{i+1}'] = df['compare'].tolist()[i]

In [74]:
pierre_de_la_vigne = list(dico_un.values())*len(viard)
cles_de_la_vigne = list(dico_un.keys())*len(viard)

jean_viard = copies(list(viard.values()))
cles_jean = copies(list(viard.keys()))


In [77]:
tqdm.pandas()
zoe = pd.DataFrame({'viard':jean_viard,'pdv':pierre_de_la_vigne})

zoe['trigram_commun'] = zoe.progress_apply(lambda row: avoir_trigram_commun(row["viard"], row["pdv"]), axis=1)
zoe['key_viard'] = cles_jean
zoe['key_pdv'] = cles_de_la_vigne
zoe = zoe[['key_viard','viard','key_pdv','pdv','trigram_commun']]
lucie = zoe[zoe['trigram_commun'] == True]
lucie = lucie.reset_index(drop=True)
print(len(lucie))

100%|█████████████████████████████████████| 23595/23595 [16:33<00:00, 23.75it/s]


1134


In [80]:
han = []
trigrams = []

for i in tqdm(range(len(lucie))):
    inter = []
    for tri in string_to_trigrams(lucie['viard'][i].split()):
        if tri in string_to_trigrams(lucie['pdv'][i].split()):
            inter.append(tri)
    han.append(len(inter))
    trigrams.append(inter)

100%|███████████████████████████████████████| 1134/1134 [01:22<00:00, 13.70it/s]


In [81]:
lucie['nombre_trigrams'] = han
lucie['trigrams'] = trigrams

In [84]:
lucie['flat'] = lucie['trigrams'].apply(flatten_tri)
lucie['flat'] = lucie ['flat'].apply(join)

In [85]:
lucie = lucie[lucie['nombre_trigrams']>1]

In [86]:
trigrams_delete = [['tam', 'presentibus', 'quam'],['notum', 'facimus', 'uniuersis'],
                  ['presentibus', 'quam', 'futuris'],['uniuersis', 'tam', 'presentibus'],
                  ['quam', 'futuris', 'quod'],['de', 'speciali', 'gratia'],
                  ['et', 'ex', 'certa'],['ex', 'certa', 'scientia'],['speciali', 'gratia', 'et'],
                  ['futuris', 'quod', 'nos'],['presentes', 'litteras', 'inspecturis'],
                   ['uniuersis', 'presentes', 'litteras'],
                  ['facimus','uniuersis','tam'],['de','regno','nostro'],
                  ['in','regno','nostro'],['et','regni','nostri'],['regni','nostri','et'],
                  ['de','speciali','gratia']]

In [87]:
def remove_lists(row):
    return [sublist for sublist in row if sublist not in trigrams_delete]

In [88]:
lucie['final_grams'] = lucie['trigrams'].apply(remove_lists)

In [89]:
lucie = lucie[lucie["final_grams"].apply(lambda x: all(sublist for sublist in x))]

In [90]:
def empty_lists(liste):
    return liste != []

In [91]:
lucie['not_empty'] = lucie['final_grams'].apply(empty_lists)

In [92]:
lucie = lucie[lucie['not_empty'] == True].reset_index(drop=True)

In [94]:
def get_len(liste):
    return len(liste)

In [95]:
lucie['len'] = lucie['final_grams'].apply(get_len)

In [99]:
lucie = lucie[lucie['len'] != 1].reset_index(drop=True)

In [114]:
resultats = []
for index, row in tqdm(lucie.iterrows()):
    try:
        resultat = test_2(row["viard"], row["pdv"])
        resultats.append(resultat)
    except Exception as e:
        print(f"Erreur survenue pour la ligne {index}: {e}")
        resultats.append(None)

lucie["resultat"] = resultats

125it [2:07:23, 61.15s/it] 


In [356]:
def extract_tri(liste):
    tri = []
    for word in liste[0]:
        if word.startswith('**'):
            tri.append(word)
    return tri
def clean_extract(liste):
    return [item.strip('**') for item in liste]
def split(sentence):
    return sentence.split()

In [293]:
lucie['trigram_post'] = lucie['resultat'].apply(extract_tri)

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [122]:
lucie['trigram_post'] = lucie['trigram_post'].apply(clean_extract)

In [126]:
lucie['trigram_pre'] = lucie['final_grams'].apply(split)

In [210]:
lucie['identical'] = lucie['trigram_post'] == lucie['trigram_pre']

In [145]:
counter_most = Counter(lucie['key_pdv'].tolist())

In [158]:
pierre = list(dict(counter_most).keys())
freq = list(dict(counter_most).values())

In [172]:
michelle = []

for i in range(len(pierre)):
    michelle.append(int(pierre[i].split('_')[1]))
len(michelle)

66

In [165]:
livre_1 = 0
livre_2 = 0
livre_3 = 0
livre_4 = 0
livre_5 = 0
livre_6 = 0

for i in range(len(michelle)):
    if michelle[i] < 33:
        livre_1 += freq[i]
    if 32 < michelle[i] < 92:
        livre_2 += freq[i]
    if 91 < michelle[i]< 179:
        livre_3 += freq[i]
    if 178 < michelle[i] < 195:
        livre_4 += freq[i]
    if 194 < michelle[i] < 331:
        livre_5 += freq[i]
    if 330 < michelle[i] < 364:
        livre_6 += freq[i]

In [175]:
print(livre_1 + livre_2 + livre_3+ livre_4+livre_5+livre_6)

125


In [176]:
livre_1 = 0
livre_2 = 0
livre_3 = 0
livre_4 = 0
livre_5 = 0
livre_6 = 0

for i in range(len(michelle)):
    if michelle[i] < 33:
        livre_1 += 1
    if 32 < michelle[i] < 92:
        livre_2 += 1
    if 91 < michelle[i]< 179:
        livre_3 += 1
    if 178 < michelle[i] < 195:
        livre_4 += 1
    if 194 < michelle[i] < 331:
        livre_5 += 1
    if 330 < michelle[i] < 364:
        livre_6 += 1

In [183]:
print(livre_1,livre_2,livre_3,livre_4,livre_5,livre_6)

9 11 10 2 21 13


In [207]:
def from_list_of_lists(liste):
    return [item for sublist in liste for item in sublist]

In [208]:
lucie['trigram_pre'] = lucie['final_grams'].apply(from_list_of_lists)

In [230]:
lucie = lucie[lucie['similar'] == False].reset_index(drop=True)

In [244]:
def extract_word_sequences(word_list, prefix):
    result = []
    current_sequence = []

    for word in word_list:
        if word.startswith(prefix):
            current_sequence.append(word)
        else:
            if current_sequence:
                result.append(current_sequence)
                current_sequence = []
    
    if current_sequence:
        result.append(current_sequence)
    
    return result

In [296]:
lucie['trigram_post'] = lucie['resultat'].apply(lambda x: extract_word_sequences(x[0],'**'))

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [297]:
def clean_asterix(liste):
    return [clean_extract(sublist) for sublist in liste]

In [298]:
lucie['trigram_post'] = lucie['trigram_post'].apply(clean_asterix)

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [299]:
lucie['trigram_post'].apply(remove_lists)[7]

[['de', 'speciali', 'gratia', 'et', 'ex', 'certa'],
 ['et', 'eius', 'heredibus']]

In [300]:
def remove_words_with_trigrams(word_lists, trigrams_to_remove):
    filtered_word_lists = []

    for word_list in word_lists:
        filtered_sublist = word_list.copy()
        for trigram in trigrams_to_remove:
            trigram_len = len(trigram)
            i = 0
            while i <= len(filtered_sublist) - trigram_len:
                if all(filtered_sublist[i + j] == trigram[j] for j in range(trigram_len)):
                    filtered_sublist[i:i + trigram_len] = []
                else:
                    i += 1
        filtered_word_lists.append(filtered_sublist)

    return filtered_word_lists

In [301]:
lucie['trigram_post'] = lucie['trigram_post'].apply(lambda x: remove_words_with_trigrams(x,trigrams_delete))

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [302]:
def remove_short_lists(list_of_lists):
    filtered_lists = [sublist for sublist in list_of_lists if len(sublist) >= 3]
    return filtered_lists

In [303]:
lucie['trigram_post'] = lucie['trigram_post'].apply(remove_short_lists)

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [304]:
lucie['similar'] = lucie['trigram_post'].apply(similar_lists)

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [305]:
def similar_lists(liste):
    if len(liste) != 2:
        return False
    return liste[0] == liste[1]

In [306]:
lucie = lucie[lucie['similar'] == False]

In [312]:
def same_sized_lists(liste):
    if len(liste) != 2:
        return False
    elif liste[0] == 3 and liste[1] == 3:
        return True

In [329]:
lucie['same_size'] = lucie['trigram_post'].apply(same_sized_lists)

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [318]:
mich = lucie[lucie['same_size'] == False].reset_index(drop=True)

In [327]:
lucie = lucie[['key_viard','viard','key_pdv','pdv','final_grams','resultat','trigram_post','trigram_pre']]

In [334]:
lucie[lucie['same_size'] == False]

,key_viard,viard,key_pdv,pdv,final_grams,resultat,trigram_post,trigram_pre,same_size
1,viard_1,philippus ualesii et andegauie comes francie e...,pdv_235,exposuit excellentiae nostrae b de fidelis no...,"[[humiliter, supplicauit, ut], [supplicauit, u...","[[philippus, ualesii, et, andegauie, comes, fr...","[[humiliter, supplicauit, ut, super, hoc]]","[humiliter, supplicauit, ut, supplicauit, ut, ...",False
4,viard_9,philippus dei gratia francorum rex notum facim...,pdv_292,pro parte o et t fidelium nostrorum nostrae fu...,"[[cum, suis, iuribus], [suis, iuribus, et]]","[[philippus, dei, gratia, francorum, rex, notu...","[[cum, suis, iuribus, et, pertinenciis]]","[cum, suis, iuribus, suis, iuribus, et]",False
5,viard_9,philippus dei gratia francorum rex notum facim...,pdv_340,regnantibus nobis feliciter et triumphantibus ...,"[[et, eius, heredibus], [gratia, et, ex]]","[[philippus, dei, gratia, francorum, rex, notu...","[[et, eius, heredibus]]","[et, eius, heredibus, gratia, et, ex]",False
6,viard_11,philippus dei gratia francorum rex notum facim...,pdv_281,ad maiestatis nostrae presentiam nuper acceden...,"[[usque, ad, summam], [usque, ad, summam], [us...","[[philippus, dei, gratia, francorum, rex, notu...","[[usque, ad, summam], [usque, ad, summam], [us...","[usque, ad, summam, usque, ad, summam, usque, ...",False
7,viard_11,philippus dei gratia francorum rex notum facim...,pdv_340,regnantibus nobis feliciter et triumphantibus ...,"[[gratia, et, ex], [et, eius, heredibus]]","[[philippus, dei, gratia, francorum, rex, notu...","[[et, eius, heredibus]]","[gratia, et, ex, et, eius, heredibus]",False
...,...,...,...,...,...,...,...,...,...
81,viard_58,philippus dei gratia francorum rex notum facim...,pdv_306,uniuersis uicariis potestatibus rectoribus ter...,"[[ex, officio, nostro], [ex, officio, nostro],...","[[philippus, dei, gratia, francorum, rex, notu...","[[ex, officio, nostro], [ex, officio, nostro],...","[ex, officio, nostro, ex, officio, nostro, ex,...",False
85,viard_61,philippus etc notum facimus uniuersis tam pres...,pdv_281,ad maiestatis nostrae presentiam nuper acceden...,"[[usque, ad, summam], [usque, ad, summam], [us...","[[philippus, etc, notum, facimus, uniuersis, t...","[[usque, ad, summam], [usque, ad, summam], [us...","[usque, ad, summam, usque, ad, summam, usque, ...",False
86,viard_62,philippus dei gratia francorum rex notum facim...,pdv_21,leuate in circuitu oculos uestros arrigite fil...,"[[statum, pristinum, et], [pristinum, et, debi...","[[philippus, dei, gratia, francorum, rex, notu...","[[statum, pristinum, et, debitum]]","[statum, pristinum, et, pristinum, et, debitum]",False
88,viard_62,philippus dei gratia francorum rex notum facim...,pdv_53,detestabile quidem et segregatum qualibet huma...,"[[et, eorum, complices], [et, eorum, complices...","[[philippus, dei, gratia, francorum, rex, notu...","[[et, eorum, complices], [et, eorum, complices...","[et, eorum, complices, et, eorum, complices, e...",False


In [339]:
def remove_identical_sublists(list_of_lists):
    if all(sublist == list_of_lists[0] for sublist in list_of_lists):
        return True
    else:
        return False

False


In [340]:
lucie['memes'] = lucie['trigram_post'].apply(remove_identical_sublists)

/home/noe/python/miniconda/envs/coursnlp/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [349]:
lucie = lucie.reset_index(drop=True)

In [350]:
lucie[lucie['memes'] == False]

,key_viard,viard,key_pdv,pdv,final_grams,resultat,trigram_post,trigram_pre,same_size,memes
0,viard_1,philippus ualesii et andegauie comes francie e...,pdv_62,curas nostras consideratio studiosa sollicitat...,"[[ut, super, hoc], [ad, hoc, per]]","[[philippus, ualesii, et, andegauie, comes, fr...","[[ut, super, hoc], [ad, hoc, per]]","[ut, super, hoc, ad, hoc, per]",None,False
2,viard_1,philippus ualesii et andegauie comes francie e...,pdv_245,imperialis beneuolentia fidelium subiectorum e...,"[[humiliter, supplicauit, ut], [in, hac, parte]]","[[philippus, ualesii, et, andegauie, comes, fr...","[[humiliter, supplicauit, ut], [in, hac, parte]]","[humiliter, supplicauit, ut, in, hac, parte]",None,False
3,viard_8,philippus dei gratia francorum rex notum facim...,pdv_350,decus est principum fidelium uotis clementer a...,"[[futuris, quod, cum], [qui, pro, tempore], [p...","[[philippus, dei, gratia, francorum, rex, **no...","[[futuris, quod, cum], [qui, pro, tempore, fue...","[futuris, quod, cum, qui, pro, tempore, pro, t...",None,False
9,viard_32,philippus dei gratia francorum rex uniuersis p...,pdv_43,ne fama preambula quae multorum uariata relati...,"[[nobis, et, successoribus], [et, successoribu...","[[philippus, dei, gratia, francorum, rex, uniu...","[[nobis, et, successoribus], [nobis, et, succe...","[nobis, et, successoribus, et, successoribus, ...",False,False
12,viard_32,philippus dei gratia francorum rex uniuersis p...,pdv_277,cum inter ceteros fideles nostros uos et ciuit...,"[[et, fidelis, noster], [de, bono, in], [bono,...","[[philippus, dei, gratia, francorum, rex, uniu...","[[et, fidelis, noster], [de, bono, in, melius]]","[et, fidelis, noster, de, bono, in, bono, in, ...",None,False
13,viard_32,philippus dei gratia francorum rex uniuersis p...,pdv_283,cum nomina castellanorum iurisdictionis tuae e...,"[[in, antea, per], [qui, nunc, sunt]]","[[philippus, dei, gratia, francorum, rex, uniu...","[[in, antea, per], [qui, nunc, sunt]]","[in, antea, per, qui, nunc, sunt]",None,False
15,viard_32,philippus dei gratia francorum rex uniuersis p...,pdv_333,opus in nobis egregium clementia suggerente pe...,"[[et, fidelis, noster], [et, ut, in]]","[[philippus, dei, gratia, francorum, rex, uniu...","[[et, fidelis, noster], [et, ut, in]]","[et, fidelis, noster, et, ut, in]",None,False
16,viard_32,philippus dei gratia francorum rex uniuersis p...,pdv_340,regnantibus nobis feliciter et triumphantibus ...,"[[redditus, usque, ad], [de, bono, in], [bono,...","[[philippus, dei, gratia, francorum, rex, uniu...","[[redditus, usque, ad], [de, bono, in, melius]]","[redditus, usque, ad, de, bono, in, bono, in, ...",None,False
17,viard_32,philippus dei gratia francorum rex uniuersis p...,pdv_357,principium epistulae fridericus duci austriae ...,"[[et, fidelis, noster], [nobis, et, successori...","[[philippus, dei, gratia, francorum, rex, uniu...","[[et, fidelis, noster], [nobis, et, successori...","[et, fidelis, noster, nobis, et, successoribus...",False,False
24,viard_36,philippus dei gratia francorum rex notum facim...,pdv_342,etsi mera beneficia principum solius gratiae l...,"[[quod, nos, attendentes], [gaudeant, et, utan...","[[philippus, dei, gratia, francorum, rex, **no...","[[quod, nos, attendentes], [gaudeant, et, utan...","[quod, nos, attendentes, gaudeant, et, utantur]",None,False


In [351]:
lucie['trigram_post'][43]

[['pacifice', 'et', 'quiete'],
 ['pacifice', 'et', 'quiete'],
 ['pacifice', 'et', 'quiete'],
 ['pacifice', 'et', 'quiete']]

In [352]:
x,y = lucie['resultat'][43]

In [354]:
print('Texte')
print('----------')
display(Markdown(' '.join(x)))
print('----------')
print('Pierre de la Vigne')
print('-----------')
display(Markdown(' '.join(y)))    
print('----------')

Texte
----------


philippus dei gratia francorum rex notum facimus uniuersistam presentibus quam futuris nos uidisse quasdam litteras sigillo nostro cera uiridi cum filis sericis sigillatas omni suspicione carentes formam que sequitur continentes philippus dei gratia francorum rex notum facimus uniuersis tam presentibus quam futuris nos litteras infrascriptas uidisse tenorem qui sequitur continentes karolus dei gratia francorum et nauarre rex notum facimus uniuersis tam presentibus quam futuris nos infrascriptas uidisse litteras in hec uerba ouz ceux qui ces présentes lettres uerront giles haquin garde de la preuosté de paris salut sachent tuit que par deuant estienne le leu dit de mante et guiart monchet clers notaires iurez de par nostre seigneur le roy ou chastellé de paris ausquels quant ces choses nous auons commis et commettons du tout nostre pooir en adioustant eux foy plainiere en ce cas et en greigneur et ce faire oir et raporter pour nous et en lieu de nous et mettre en fourme publique especiaument de par nous enuoiez et destinez pour ce furent presenz en propres parsonnes noble homme monseigneur nicole de ioynuille cheualier madame philippe sa famme fille de monseigneur iehan fourré cheualier et damoiselle ysabel la fourrée affermerent en bonne uerité que il paisiblement seul et pour le tout du propre heritaige desdites dame et damoiselle auoient prenoient receuoient et possessoient chascun an le iour de la saint remy dis soulz parisis de menu cens annuel et perpetuel sus et en plusieurs maisons et places sus et en la proprieté et treffons diceles sus et en tous leur appartenances et appendances tenues de eux estans et seans en leur terre et seignorie paris qui est appellée la terre de therouenne lesquelles maisons et places sont aus confreres de la confrarie de saint iaques aus pelerins de paris et appliquées et appropriées de par yceux confreres la fondation et edifiement de lospital de saint iaques de paris appartenant ausdiz confreres nouuellement establi et fondé de par yceux confreres pour eux et pour leurs successeurs en lonneur de dieu et de monseigneur saint iaques seant paris en la grant rue près de la porte saint denis si comme tout se comporte de la maison de ardoise iusques la rue de mauconseil et du coing dicele rue et en ycele iusques la maison lorenz préuost et aboutissans par derrieres aus murs de ladite maison lorenz préuost touz lesquels diz solz de menu cens auecques touz les droiz toutes les seignouries proprietez possessions saisines et toutes les actions reeles et personneles mixtes directes taisibles et expresses et toutes autres que yceux monseigneur nicole madame philippe sa famme et damoiselle ysabel tant conioinctement comme diuiseement ou auoient ou entendoient auoir et es lieux dessusdiz et en chascun et enuers toutes personnes pour raison de tout ce comment que ce soit ou fust et en toutes leur appartenances et appendances mouuanz en fié du conte de dampmartin lesdiz monseigneur nicole madame philippe sa famme et damoisele ysabel et meesmement ycele dame de lauttorité ele donnée souffisamment en toutes choses dudit monseigneur nicole son mari en la presence desdiz iurez pour faire acorder entrimer et aemplir toutes les choses et chascune ci dessus et dessoubz escriptes de leur bon grez et pour leur commun prouffit chascun pourtant comme il li touche et puet touchier uendirent cesserent transportèrent et quittèrent frans et quites tous amortis de eux de leurs hoirs et de leur successeurs tant seulement par deuant lesdiz clers et recognurent eux auoir uenduz et amortiz cessé transporté quité ottroié et delessié par nom de pure simple et perpetuel uente et amortissement frans et quittes touz amortiz si comme dit est desorendroit touz iours aus deuanz diz confieres achatans pour eux et pour leur successeurs ou nom dudit hospital auoir tenir et par droit de heritaige poursuiuir paisiblement franchement et quitement touz amortiz si comme dit est et en main morte senz riens excepter retenir y ne reclamer dores en auant et sanz ce que yceux uendeurs leurs hoirs ne leurs successeurs nulz iours les puissent contraindre uendre les ou mettre hors de leur main ou faire en eux aucune finance queleque ele soit cest assauoir tout pour le pris de deux cenz liures de bons petiz tournois fors que lesdiz uendeurs confesserent en auoir eu et receu leurs frans et quites en bonne monnoie bien comptée et nombrée desdiz achateurs ou de leur commandement et dont il se tindrent bien paiez et aggreez entérinement par deuant lesdiz clers iurez et diceli pris il quittèrent touz iours lesdiz achateurs ceux qui ont et auront cause de eux et touz autres cui quittence en appartient et est assauoir que entre les maisons et places dessusdites sont enclauées les maisons ci dessouz nommeez lesquelles sont et demeurent sauues ausdiz uendeurs et leurs hoirs et en leur censiue et seignourie de leurdite terre de therouenne senz amortissement cest assauoir les maisons iehan toussac iehan de la nasse entretenans les maisons rogier luissier la maison qui fut iehan de lusarches qui est present audit iehan toussac les maisons mahieu de biauuez les maisons guillaume de fontenay la maison iaques coquet la maison estienne le bariller et les maisons pierre le moleur tant seulement si comme il disoient et parmi ledit pris lesdiz uendeur et memeement ladite dame ladite auttorité leur hoirs et leur successeurs ceux qui ont et auront cause de eux chascun pour tant comme il li touche et puet touchier sont et seront tenuz garantir déliurer et deffendre toutes les choses et chascune dessus uendues franches et quites toutes amorties et en main morte si comme dit est leurs propres couz perilz et despens toutesfoiz et quantes quil en seront requis du porteur de ces lettres senz autre procuration porter de toutes charges exactions debtes obligacions de touz troubles et empeschemenz ausdiz acheteurs et leurs successeurs en iugement et dehors contre touz et enuers touz perpetuelment et auec ce rendre et paier touz couz dommages mises despens et interez qui faiz et encouruz seroient par defaute de leur garantie non portée comme dit est desquels et sur lesquels il uoudrent et acorderent que le porteur de ces lettres soit creuz par son simple serement senz autre preue faire sauf ausdiz uendeurs et leurs hoirs que il ne prennent de riens en main de amortir ladite uente enuers le conte de dampmartin ne enuers les autres seigneurs par dessus et promistrent lesdiz uendeurs par leur fois bailliés corporelment es mains desdiz iurez et par leur seremenz faiz aus sainz euuangiles que il toutes ces choses pour tant comme chascun touche et puet touchier tendront et empliront et que contre ne uendront procurront ne souffreront faire uenir par aucune maniere nul iour par eux ne par autres et quant toutes les choses dessusdites et chascune tenir et aemplir en la maniere que dit est lesdiz uendeurs chascun pour tant comme il li touche et puet touchier ont obligié et souzmis especiaument senz aucune exception de droit ou de fait eus leurs hoirs et leur successeurs touz leurs biens et les biens de leur hoirs et de leur successeurs meubles et non meubles resenz et auenir ou que il soient et puissent estre trouuez tant pour saisir arrester exploitier leuer uendre et despendre tel feur tele uente senz aucun respit ou delay par nous et par noz successeurs preuoz de paris ou par quelconque autre iustice que le porteur de ces lettres uoudra eslire pour loyal garantie porter touz iours de ceste présente et iusques la quantité de ce que ces lettres soient ou fussent du tout enterinées et empliées en la maniere que dit est renunçanz en ce fait lesdiz uendeurs etc nous la relacion dyceux auons mis en ces présentes lettres le seel de la preuosté de paris lan de grâce mil ccc uint et un le derrenier iour dauril ueille de la feste saint iasques et saint philippe apostres item quasdam alias in hec uerba touz ceux qui ces présentes lettres uerront renaut conte de dampmartin seigneur de trie et de moucy salut corne les freres et les suers de lospital de saint iaques nouuelement establiz et fondez paris seant en la grant rue saint denis entre la porte et leglise de saint leu et de saint gile nous eussent requis que diz souls de parisis de menu cens ou de fons de terre annuele et perpetuele que monseigneur nicole de ioynuille cheualier madame philipe sa famé fille de monseigneur iehan fourré cheualier et dameseille ysabel la fourrée cousine germaine dudit monseigneur iehan prenoient chascun an la feste saint remy du propre héritage desdites dames et damoiseille du fié que il tiennent de nous paris apelé la terre de therouenne ouquel lesdiz mariez auoient les trois pars de ladite damoisele la quarte sus ledit hospital et sus plusieurs maisons enclauées entre la maison dardoise et la rue de mauconseil lesquels diz soûls yceux mariez et dameisele auoient nouuelement uenduz touz amortiz de eux et de leur successeurs tant seulement ausdiz freres et suers et leurs successeurs que nous ladite uente uousissiens confermer et amortir en tant comme nous appartenoit pour ce que tenue est de nous nu nu et aussi le tenions nu à nu du roy de france sachent tuit que nous pour auoir participation aus biens fais et oroisons dudit hospital et parmi cinquante liures parisis que nous conffessons auoir euz et receuz desdiz freres et suers dont nous nous tenons bien paiez la requeste desdiz freres et suers yceux dix solz auec touz les drois quels que il soient que nous auions comment que ce fust en tant comme nous appartient amortissons desorendroit perpetuelment et en ladite uente faite desdiz mariez et damoisele approuuant et confirmant uoulons que yceux freres et suers et leurs successeurs les tiegnent touz iours franchement come amortiz senz que eux ne leur successeurs soient tenuz dore en auant faire en finance ne redeuance nous ne noz successeurs et promettons en bonne foy déliurer et garantir leur ledit amortissement de nostre fait qui nous touche et puet touchier tant seulement contre touz noz couz si tost que requis en serons desorendroit touz iours perpetuelment sur lobligacion de touz noz biens et de noz successeurs en tesmoing de ce nous auons fait seeler ces présentes lettres de nostre propre seel donné lan de grâce mil ccc uint et un le secont iour de may nos autem progenitorum nostrorum qui ad diuini cultum numinis piaque et religiosa loca indefessis animis igitur aspirasse noscuntur uestigia quantum cum deo possumus imitantes bonorumque et quaritatis operum que futuris temporibus in hospitali predicto fieri domino inspirante speramus participes effici cupientes et ut diuine nobis propiciacionis gloriosique apostoli christi iacobigratiam acquiramus uendicionem quittacionem cessionem translacionem confirmacioncmque predictas et omnia alia et singula in suprascriptis contenta litteris uolentes laudantes approbantes et ratificantes ea omnia et singula tenore presencium auctoritate nostra regia de speciali gratia et ex certa scientia confirmamus eidem quoque hospitali fratribus et sororibus eiusdem gratiam facere uolentes pleniorem concedimus quod ipsi in feodis retrofeodis censiuis uel allodiis nostris donacionis empcionis uel alio quocumque iusto titulo centum libras parisiensium annui et perpetui redditus acquirere ualeant absque tamen omni iusticia acquisitasque tenere et possidere **pacifice** **et** **quiete** una cum rebus et possessionibus prescriptis iam acquisitis sicut premittitur absque coactione uendendi uel extra manum suam ponendi premissa uel aliqua premissorum seu prestandi pro eis nobis uel quibuscumque nostris successoribus financiam qualemcumque nostro tamen in aliis et alieno in omnibus iure saluo quod ut perpetue stabilitatis robur obtineat presentes litteras sigilli nostri fecimus impressione muniri actum et datum parisius anno domini mo ccco uicesimo primo mense marcio cum igitur nobis ex parte confratrum hospitalis sancti iacobi parisiensis fuerit supplicatum quod de centum libris parisiensium annui et perpetui rcdditus quem ex concessione inclite memorie karoli quondam regis francorum et nauarre domini nostri carissimi in ipsius regis feodis retrofeodis censiuis uel allodiis donacionis empcionis uel alio quocumque iusto titulo absque tamen iusticia poterant acquirere et acquisitum tenere ac possidere **pacifice** **et** **quiete** absque coactione uendendi uel extra manum suam ponendi seu prcstandi de redditu ipso sibi uel successoribus suis ifnanciam qualemcumque quadraginta libras parisiensium annui redditus acquisiti seu acquirendi per decanum et capitulum ecclesie sancti germani autissiodorensis parisius caderent in utilitatem ipsius ecclesie conuertende et quod dicte regie concessionis gratia dictis facta confratribus de predictis centum libris parisiensium ut predicitur acquirendis tam in dicte ecclesie quo ad quadraginta libras parisiensium annui redditus quam in dictorum confratrum ac hospitalis sui predicti quo ad sexaginta libras parisiensium annui redditus que restabant comodum per dimisionem huiusmodi redundaret nos ipsorum confratrum supplicacionem fauore beniuolo in hac parte uolentes prosequi dictis decano et capitulo per alias litteras nostras concessimus quod ipsi successoresque sui suo et ecclesie sue predicte nomine uiginti septem arpenta uel circa terre cultibilis acquisita per eos apud atteinuilla triginta libras parisiensium annui redditus uel circiter extimata ualere communiter necnon decem libras annui redditus acquirendi per ipsos sub modo et forma pretactis tenere et perpetuo possidere ualeant ut in aliis litteris nostris plenius continetur predictisque confratribus ut uirtute gratie sibi facte per dicti domini nostri litteras suprascriptas quarum originale duximus retinendum omnia nichilominus alia superius comprehensa in ipsis litteris approbantes laudantes et auctoritate regia confirmantes concedimus per presentes quod ipsi confratres sexaginta libras parisiensium annui redditus ut premittitur acquirere et acquisitum tenere ac possidere **pacifice** **et** **quiete** ualeant absque coactione uendendi uel extra manum suam ponendi seu prestendi de ipso nobis uel successoribus nostris financiam quelemcumque quod ut ratum et stabile perseueret fecimus nostrum presentibus apponi sigillum actum parisius anno domini m°ccc° uicesimo octauo mense februarii sane cum fratres hospitalis sancti iacobi suprascripti quatuor libras turonensium annui redditus qui olim ex ordinatione testamenti alfonsi quondam comitis pictauensis et tholose heredibus defuncti luce coci seruitoris ipsius comitis super emolumentis prepositure bellimontis supra ysaram ex una parte et alias quatuor libras turonensium annui redditus qui ex ordinacione ipsius testamenti super dictis emolumentis ipsius prepositure heredibus defuncti roberti dicti cauz de gonnessa eiusdem comitis seruitoris per executores testamenti predicti diuisi et assignati fuerunt iusto titulo seu iustis titulis ad opus ipsius hospitalis acquisiisse noscantur nos uolumus tenore presencium concedentes quod fratres ipsius hospitalis tam presentes quam futuri dictas octo libras turonensium annui et perpetui redditus super dicta prepositura bellimontis in deductionem sexaginta librarum parisiensium annui redditus quas eis ut suprascriptum est licebat acquirere tenere participere et habere ualeant perpetuo **pacifice** **et** **quiete** absque feodo et iusticia qualicumque ita quod dictas octo libras redditus annui uendere uel extra manum suam ponere uel financiam qualemcumque nobis uel successoribus nostris propter hoc prestare uel facere non cogantur premissis omnibus et singulis in suprascriptis litteris ut premittitur contentis quo ad alia omnia et singula in suo robore perpetuo ualituris quas quidem litteras in camera compotorum parisius retineri fecimus cancellatas pro eo quod dicte octo libre turonensium annui redditus de summa predicta sexaginta librarum parisiensium deducuntur que ut firma et stabilia perpetuo permaneant etc actum parisius anno domini mo ccco quadragesimo mense mayo per gentes compotorum matheus sine financia milo

----------
Pierre de la Vigne
-----------


conquerente i de fideli nostro nostra serenitas intellexit quod cum ipse **pacifice** **et** **quiete** teneret in territorio castri de loco quasdam suas possessiones ad eum rationabiliter pertinentes l de mulier destituit eum absentem et in nostris seruitiis sicut asserit existentem possessionibus antedictis iuris ordine non seruato super quo cum supplicauerit sibi per nostram excellentiam secundum iustitiam subueniri ipsius supplicationibus uelut iustitiam continentibus inclinati fidelitati tuae precipiendo mandamus quatenus si tibi constiterit ita esse possessiones ipsas muliere prefata predicto fideli nostro restitui facias ut est iustum ne iustam super hoc causam habeat conquerendi

----------
